Passo 1: importar bibliotecas

In [1]:
import pandas as pd
from pathlib import Path
import os

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.layout import LAParams
from pdfminer.converter import TextConverter
from io import StringIO
from pdfminer.pdfpage import PDFPage


Passo 2: Localizar o diretório onde se encontram as pastas das medições semanais das DACTEs e NFs

In [2]:
medicao= 'automacoes_notas'  #Atualizar a pasta antes de rodar o código
caminho = Path(f'{medicao}')
tecnolog= r'C:\Users\roger'
medicao_1= r'\\digicorner.intra\br_fileserver\Departamentos_M32\2362-Supply Chain\2332-Supply Chain JV DOFCON\Geral\21. Contratos\1. Categorias\2020\5. Transporte Fracionado - TECNOLOG'
print('Operação realizada')


Operação realizada


In [3]:
os.chdir(tecnolog)
os.getcwd()


'C:\\Users\\roger'

Passo 3: listar os arquivos da pasta

In [7]:
arquivos = caminho.iterdir()

all_files= []

i=0
for arquivo in arquivos:
    a= str(arquivo)
    all_files.append(a[17:])
    print(a[17:])
print(i)


10769-1.pdf
1281-1.pdf
15452-1.pdf
17296-0.pdf
19751-1.pdf
CTE_002764.pdf
CTE_035816.pdf
CTE_035856.pdf
CTE_036161.pdf
CTE_036288.pdf
0


In [ ]:
10260-1.pdf
10267-1.pdf
116991-1.pdf
116992-1.pdf
132410.pdf
167132-25.pdf
17463-1.pdf
17465-1.pdf
17466-1.pdf
17467-1.pdf
17472-1.pdf
18939-1.pdf
23041-1.pdf
262955-1.pdf
36913-1.pdf
698216-3.pdf
84412-3.pdf
91005-1.pdf
95700-1.pdf

In [18]:
Passo 4: Ativar o modo leitura dos arquivos PDF

In [8]:
def get_pdf_file_content(path_to_pdf):
    
    '''
    path_to_pdf: is the parameter that will give access to the PDF File 
    we want to extract the content.
    '''
    
    '''
    PDFResourceManager is used to store shared resources such as fonts or images that 
    we might encounter in the files. 
    '''
    
    resource_manager = PDFResourceManager(caching=True)
    
    '''
    create a string object that will contain the final text the representation of the pdf. 
    '''
    out_text = StringIO()
    
    '''
    UTF-8 is one of the most commonly used encodings, and Python often defaults to using it.
    In our case, we are going to specify in order to avoid some encoding errors.
    '''
    codec = 'utf-8'
    
    """
    LAParams is the object containing the Layout parameters with a certain default value. 
    """
    laParams = LAParams()
    
    '''
    Create a TextConverter Object, taking :
    - ressource_manager,
    - out_text 
    - layout parameters.
    '''
    text_converter = TextConverter(resource_manager, out_text, laparams=laParams)
    fp = open(path_to_pdf, 'rb')
    
    '''
    Create a PDF interpreter object taking: 
    - ressource_manager 
    - text_converter
    '''
    interpreter = PDFPageInterpreter(resource_manager, text_converter)

    '''
    We are going to process the content of each page of the original PDF File
    '''
    for page in PDFPage.get_pages(fp, pagenos=set(), maxpages=0, password="", caching=True, check_extractable=True):
        interpreter.process_page(page)

    '''
    Retrieve the entire contents of the “file” at any time 
    before the StringIO object’s close() method is called.
    '''
    text = out_text.getvalue()

    '''
    Closing all the ressources we previously opened
    '''
    fp.close()
    text_converter.close()
    out_text.close()
    
    '''
    Return the final variable containing all the text of the PDF
    '''
    return text


Passo 5: Recertificar se o código está lendo o diretório corretamente

In [9]:
os.chdir(caminho)
os.getcwd()


'C:\\Users\\roger\\automacoes_notas'

* Passo 6: Localizar o(s) número(s) da(s) Nota(s) Fiscal(is) dentro do(s) DACTE(s) e agrupá-los em tuplas dentro de uma lista

In [10]:
#NÃO ESQUECER DE A,TUALIZAR A LISTA COM OS CTe's
lista=['CTE_002764.pdf',
'CTE_035816.pdf',
'CTE_035856.pdf',
'CTE_036161.pdf',
'CTE_036288.pdf']

df_Cte_NF= pd.DataFrame()
Cte=[]
#NF= []
#lista_Cte_NF= []

t=0
for i in lista:


    nota= get_pdf_file_content(i)
    loc= nota.find('NF - SÉRIE')
    Cte.append((i,nota[loc+10:loc+20]))
    
    t=t+1
    
print(Cte)
print(t)


[('CTE_002764.pdf', '\n000017296'), ('CTE_035816.pdf', '\n000019751'), ('CTE_035856.pdf', '\n000001281'), ('CTE_036161.pdf', '\n000015452'), ('CTE_036288.pdf', '\n000010769')]
5


In [ ]:
Passo 7: Editar nomenclatura das notas fiscais

In [12]:
#Criar uma lista de tuplas onde cada par representa o Nº da NF(lida nos ctes) e Nº da NF(lida na listagem de medicao)

all_files[:] 
NF_comparar= []
#NF_completa= []

for i in all_files[:5]:  #NÃO ESQUECER DE ATUALIZAR O RANGE DA LISTA ALL_FILES
    x=str('\\')
    posicao= i.find(x)
    posicao_2= i.find('-')
    c= i[posicao+1:posicao_2]
    posicao_2= i.find('-')
    #print(c)
    NF_comparar.append((c,i[posicao+1:]))
    #NF_completa.append(i[posicao+1:])
    
print(NF_comparar)
print(len(NF_comparar))


[('10769', '10769-1.pdf'), ('1281', '1281-1.pdf'), ('15452', '15452-1.pdf'), ('17296', '17296-0.pdf'), ('19751', '19751-1.pdf')]
5


contatenar o Nº do CTe com o Nº da NF se a NF lida no CTe tiver dispovínel na lista de medicao

Comparar as tuplas de par (NFs editado, NF original) com as tuplas de (CTes, NFs). Se o dado da posição a esquerda do par da tupla(NFs editado, NF original) for diferente do dado à direita do par da tupla (CTes, NFs), fazer a comparação com o próximo par do looping for. Se for igual, incrementar 'NF_' a frente no numero da nota. Desse modo, cada posição da lista "nome final" será composta com o par de nomenclatura CTE_XXXXXX.pdf NF_yyyyyy. 

In [13]:
nome_final=[]

contador=0
for i in NF_comparar:
    for k in Cte:
        j=0
        if not i[0] in k[1]: 
            j=j+1
        else:
            print(k[0],f'NF_{i[0]}')
            #Cte=(Cte,NF)
            nome_final.append((k[0],f'NF_{i[0]}'))
            contador = contador+1
            
print(contador) 


CTE_036288.pdf NF_10769
CTE_035856.pdf NF_1281
CTE_036161.pdf NF_15452
CTE_002764.pdf NF_17296
CTE_035816.pdf NF_19751
5


In [14]:
print(Path.cwd())


C:\Users\roger\automacoes_notas


Passo 8: Renomear as NFs com o par de informações(Nº_CTe, Nº_NF)

In [15]:

s=0
v=0
for i in NF_comparar:
    for k in Cte:
        j=0
        if not i[0] in k[1]:
            j=j+1
            #print('if')
            v=v+1
            
        else:
            #print('else')
            q= str(k[0]+' '+'NF_'+f'{i[1]}')
            Path(f'{i[1]}').rename(f'{q}')
            print(f'{i[1]}', f'{q}')
            s=s+1

print("Operação concluída")
print(v, "if")
print(s, "else")


10769-1.pdf CTE_036288.pdf NF_10769-1.pdf
1281-1.pdf CTE_035856.pdf NF_1281-1.pdf
15452-1.pdf CTE_036161.pdf NF_15452-1.pdf
17296-0.pdf CTE_002764.pdf NF_17296-0.pdf
19751-1.pdf CTE_035816.pdf NF_19751-1.pdf
Operação concluída
20 if
5 else


'CTE_160791.pdf NF_50903'